# CatBoost을 활용한 베이스라인

In [103]:
import pandas as pd
import os
import random
import numpy as np

## 1. 데이터 로딩

In [104]:
data_dir = '/opt/ml/input/data' # 경로는 상황에 맞춰서 수정해주세요!
csv_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터는 대회홈페이지에서 받아주세요 :)

In [105]:
raw_data = pd.read_csv(csv_file_path) 
raw_data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


## 2. Feature Engineering

In [170]:
import datetime

def feature_engineering(df):
    
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']
    
    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용
    correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
    correct_t.columns = ["test_mean", 'test_sum']
    correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
    correct_k.columns = ["tag_mean", 'tag_sum']

    df = pd.merge(df, correct_t, on=['testId'], how="left")
    df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    
    df.insert(3, "solveTime", np.NaN)
    train_np = df.to_numpy()

    idx_list = list()

    for i in range(len(train_np)-1):
        # 현재 문제의 timestamp을 가져온다
        current_date_data, current_time_data = train_np[i][5].strip().split(" ")
        current_year, current_month, current_day = map(int, list(current_date_data.split("-")))
        current_hour, current_minute, current_second = map(int, list(current_time_data.split(":")))

        # 다음 문제의 timestamp를 가져온다.
        next_date_data, next_time_data = train_np[i+1][5].strip().split(" ")
        next_year, next_month, next_day = map(int, list(next_date_data.split("-")))
        next_hour, next_minute, next_second = map(int, list(next_time_data.split(":")))


        # 같은 유저가 다음 문제도 같은 시험지를 풀고 았거나, 
        # 다른 시험 문제지를 같은 날짜에 풀었을 경우
        # 문제 푸는 시간 = 다음 문제가 시작 시간 - 현재 문제가 시작한 시간
        if train_np[i][0]==train_np[i+1][0] and \
            (train_np[i][2]==train_np[i+1][2] or (train_np[i][5]!=train_np[i+1][5] and current_date_data == next_date_data)):         
            train_np[i][3] = datetime.datetime(next_year, next_month, next_day, next_hour, next_minute, next_second) - datetime.datetime(current_year, current_month, current_day, current_hour, current_minute, current_second)
            train_np[i][3] = train_np[i][3].total_seconds() # 초로 변환
            #if train_np[i][3]>150 : train_np[i][3] = 150

        else :
            # 마지막으로 푼 문제인 경우는 60으로 통일
            train_np[i][3] = 60.0

        df.iloc[i,3] = train_np[i][3]

        #if train_np[i][3] == 0  : idx_list.append(i)
   
    train_np[-1][3] = 60.0
    df.iloc[-1,3] = train_np[i][3]

    
    test_mean_solveTime = df.groupby('testId')['solveTime'].mean()
    test_mean_solveTime.columns = ['test_mean_solveTime']
    # user_mean_solveTime = df.groupby('userID')['solveTime'].mean()
    # user_mean_solveTime.columns = ['user_mean_solveTime']
    tag_mean_solveTime = df.groupby('KnowledgeTag')['solveTime'].mean()
    tag_mean_solveTime.columns = ['tag_mean_solveTime']
    #assessmentItem_mean_solveTime = df.groupby('assessmentItemID')['solveTime'].mean()
    #assessmentItem_mean_solveTime.columns = ['assessmentItem_mean_solveTime']

    df = pd.merge(df, test_mean_solveTime, on=['testId'], how="left")
    #df = pd.merge(df, user_mean_solveTime, on=['userID'], how="left")
    df = pd.merge(df, tag_mean_solveTime, on=['KnowledgeTag'], how="left")
    #df = pd.merge(df, assessmentItem_mean_solveTime, on=['assessmentItemID'], how="left")

    df.insert(1, "testType", np.NaN)
    df.insert(2, "testID", np.NaN)
    df.insert(3, "questionID", np.NaN)

    train_np = df.to_numpy()


    for i in range(len(df)):
        assessmentItemID = train_np[i][4]
        train_np[i][1] = int(assessmentItemID[2])
        train_np[i][2] = int(assessmentItemID[4:7])
        train_np[i][3] = int(assessmentItemID[8:])

    df = pd.DataFrame(train_np, columns=df.columns)

    
    #df = df.drop(idx_list, axis=0)
    #df = df.drop(['solveTime'], axis=1)

    return df

In [171]:
df = pd.read_csv(csv_file_path) 
df = feature_engineering(df)
#df = feature_engineering(df).drop(columns=['Timestamp'])
df

KeyError: 'Column not found: solveTime'

In [151]:
col = list(df.columns)
col[-2] = 'test_mean_solveTime'
col[-1] = 'tag_mean_solveTime'
df.columns = col 
df

,userID,testType,testID,questionID,assessmentItemID,testId,solveTime_x,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc,test_mean,test_sum,tag_mean,tag_sum,test_mean_solveTime,tag_mean_solveTime
0,0,6,1,1,A060001001,A060000001,3.0,1,2020-03-24 00:17:11,7224,NaN,0,NaN,0.947683,1268,0.955022,637,471.632287,20.58021
1,0,6,1,2,A060001002,A060000001,8.0,1,2020-03-24 00:17:14,7225,1.0,1,1.0,0.947683,1268,0.913187,3040,471.632287,480.769901
2,0,6,1,3,A060001003,A060000001,7.0,1,2020-03-24 00:17:22,7225,2.0,2,1.0,0.947683,1268,0.913187,3040,471.632287,480.769901
3,0,6,1,4,A060001004,A060000001,7.0,1,2020-03-24 00:17:29,7225,3.0,3,1.0,0.947683,1268,0.913187,3040,471.632287,480.769901
4,0,6,1,5,A060001005,A060000001,11.0,1,2020-03-24 00:17:36,7225,4.0,4,1.0,0.947683,1268,0.913187,3040,471.632287,480.769901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,3,71,5,A030071005,A030000071,60.0,0,2020-06-05 06:50:21,438,1.0,4,0.25,0.66259,921,0.689706,2814,461.284173,523.940441
2266582,7441,4,165,1,A040165001,A040000165,11.0,1,2020-08-21 01:06:39,8836,1.0,5,0.2,0.655109,718,0.697874,2199,711.779197,563.497302
2266583,7441,4,165,2,A040165002,A040000165,46.0,1,2020-08-21 01:06:50,8836,2.0,6,0.333333,0.655109,718,0.697874,2199,711.779197,563.497302
2266584,7441,4,165,3,A040165003,A040000165,73.0,1,2020-08-21 01:07:36,8836,3.0,7,0.428571,0.655109,718,0.697874,2199,711.779197,563.497302


## 3. Train/Test 데이터 셋 분리

In [152]:
# train과 test 데이터셋은 사용자 별로 묶어서 분리를 해주어야함
random.seed(42)
def custom_train_test_split(df, ratio=0.7, split=True):
    
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)
    
    max_train_data_len = ratio*len(df)
    sum_of_train_data = 0
    user_ids =[]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        user_ids.append(user_id)


    train = df[df['userID'].isin(user_ids)]
    test = df[df['userID'].isin(user_ids) == False]

    #test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test['userID'] != test['userID'].shift(-1)]
    return train, test

In [165]:
# 유저별 분리
train, test = custom_train_test_split(df)

# 사용할 Feature 설정
FEATS = ['KnowledgeTag', 'user_correct_answer', 'user_total_answer', 
         'user_acc', 'test_mean', 'test_sum', 'tag_mean','tag_sum', 'userID','testType', 'testID', 'questionID']

# X, y 값 분리
y_train = train['answerCode']
train = train.drop(['answerCode'], axis=1)

y_test = test['answerCode']
test = test.drop(['answerCode'], axis=1)

In [166]:
#!pip install lightgbm

In [167]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np

In [168]:
lgb_train = lgb.Dataset(train[FEATS], y_train)
lgb_test = lgb.Dataset(test[FEATS], y_test)

## 4. 훈련 및 검증

In [169]:
model = lgb.train(
    {'objective': 'binary'}, 
    lgb_train,
    valid_sets=[lgb_train, lgb_test],
    verbose_eval=100,
    num_boost_round=500,
    early_stopping_rounds=100
)

preds = model.predict(test[FEATS])
acc = accuracy_score(y_test, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_test, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: KnowledgeTag, user_correct_answer, user_total_answer, user_acc, test_mean, test_sum, tag_mean, tag_sum, userID

In [138]:
# INSTALL MATPLOTLIB IN ADVANCE
# _ = lgb.plot_importance(model)

## 5. Inference

In [139]:
# LOAD TESTDATA
test_csv_file_path = os.path.join(data_dir, 'test_data.csv')
test_df = pd.read_csv(test_csv_file_path)

# FEATURE ENGINEERING
test_df = feature_engineering(test_df)

# LEAVE LAST INTERACTION ONLY
test_df = test_df[test_df['userID'] != test_df['userID'].shift(-1)]

# DROP ANSWERCODE
test_df = test_df.drop(['answerCode'], axis=1)

/tmp/ipykernel_24901/10389462.py:73: FutureWarning: Passing 'suffixes' which cause duplicate columns {'solveTime_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, assessmentItem_mean_solveTime, on=['assessmentItemID'], how="left")


In [147]:
# MAKE PREDICTION
total_preds = model.predict(test_df[FEATS])

In [148]:
# SAVE OUTPUT
output_dir = 'output/'
write_path = os.path.join(output_dir, "submission_lgbm.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
with open(write_path, 'w', encoding='utf8') as w:
    print("writing prediction : {}".format(write_path))
    w.write("id,prediction\n")
    for id, p in enumerate(total_preds):
        w.write('{},{}\n'.format(id,p))

writing prediction : output/submission_lgbm.csv
